In [ ]:
!pip install torchinfo
!pip install datasets
!pip install vit-pytorch
!pip install x-transformers
!pip install timm

In [ ]:
import os
import pprint

import datasets
import numpy as np
import timm
import torch
import torchinfo
import torchvision
import tqdm
from vit_pytorch.efficient import ViT  # 事前学習されていないバニラVisionTransformerモデルのようなもの
import x_transformers  # Transformer層をモジュールにしたようなもの

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

**データ収集(今回はデータとして、HuggingFaceにある正解ラベル付きデータを使う)**

In [ ]:
# HuggingFaceにある犬猫写真分類の正解ラベル付きDatasetDictをロード
hugging_face_dataset_dict = datasets.load_dataset("cats_vs_dogs")
# ロードしたDatasetDictからDataset(DatasetDictにあるDatasetはtrainのみ)を取得
dataset = hugging_face_dataset_dict["train"]
# Dataset内のデータのタイプを確認
print(dataset.features)
# Dataset内のデータをシャッフル
dataset_shuffle = dataset.shuffle()

In [ ]:
# Dataset内の最初の3つのデータの写真を確認
for i in range(3):
    display(dataset_shuffle[i]["image"])

In [ ]:
# Dataset内のデータを分割して、それぞれのDataset名がtrainとvalidationとtestのDatasetDictにする
train_valid_test_dataset_dict = dataset_shuffle.train_test_split(test_size=0.1,
                                                                 shuffle=True,
                                                                 stratify_by_column="labels",  # stratify_by_columnに指定したカラムのデータが均等に分かれるように分割される
                                                                 seed=42)
test_dataset = train_valid_test_dataset_dict.pop("test")
train_valid_test_dataset_dict = train_valid_test_dataset_dict["train"].train_test_split(test_size=0.2,
                                                                                        shuffle=True,
                                                                                        stratify_by_column="labels",  # stratify_by_columnに指定したカラムのデータが均等に分かれるように分割される
                                                                                        seed=42)
valid_dataset = train_valid_test_dataset_dict.pop("test")
train_valid_test_dataset_dict["validation"] = valid_dataset
train_valid_test_dataset_dict["test"] = test_dataset

In [ ]:
train_valid_test_dataset_dict

DatasetDict({
    train: Dataset({
        features: ['image', 'labels'],
        num_rows: 16855
    })
    validation: Dataset({
        features: ['image', 'labels'],
        num_rows: 4214
    })
    test: Dataset({
        features: ['image', 'labels'],
        num_rows: 2341
    })
})

**データ拡張の準備**

In [ ]:
# trainをデータ拡張するCompose(torchvision.transformsの複数の処理を一つに纏めて、末尾の処理の出力を戻り値とする)インスタンスを作成
train_argumentation = torchvision.transforms.Compose([torchvision.transforms.Resize(size=(224, 224)),  # 画像を指定の大きさにする # 単独で使う際はResizeクラスをインスタンス化してforwardと同じ使い方
                                                      torchvision.transforms.RandomResizedCrop(size=(224, 224)),  # データ拡張(DataAugmentation)として、画像をランダムに切り抜いた後に指定の大きさにする
                                                      torchvision.transforms.RandomHorizontalFlip(p=0.5),  # データ拡張として、画像をランダムに左右反転させる
                                                      torchvision.transforms.ToTensor()])  # PILやnumpyを、(Channel, Height, Width)の形で、値は0.0～1.0のFloatTensorにする
# validationをデータ拡張するComposeインスタンスを作成
valid_argumentation = torchvision.transforms.Compose([torchvision.transforms.Resize(size=(256, 256)),
                                                      torchvision.transforms.CenterCrop(size=(224, 224)),  # 画像の中央を指定の大きさで切り抜く
                                                      torchvision.transforms.ToTensor()])
# testをデータ拡張するComposeインスタンスを作成
test_argumentation = torchvision.transforms.Compose([torchvision.transforms.Resize(size=(256, 256)),
                                                     torchvision.transforms.CenterCrop(size=(224, 224)),
                                                     torchvision.transforms.ToTensor()])

**前処理(データ拡張も含む)**

In [ ]:
# Datasetクラスを作成
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, img_data_list, label_value_list, argumentation_method=None):
        self.img_data_list = img_data_list
        self.label_value_list = label_value_list
        self.argumentation_method = argumentation_method  # データ拡張のために作成したComposeインスタンスを入れる

    def __len__(self):
        return len(self.img_data_list)

    def __getitem__(self, idx):
        img_data = self.img_data_list[idx]
        argumentation_img_data = self.argumentation_method(img_data)  # インプットをPIL画像として、データ拡張のComposeインスタンスの処理を行う
        label_value = self.label_value_list[idx]
        return argumentation_img_data, label_value

In [ ]:
print(type(train_valid_test_dataset_dict["train"][:7]["image"]))
pprint.pprint(train_valid_test_dataset_dict["train"][:7]["image"])
print(type(train_valid_test_dataset_dict["train"][:7]["labels"]))
print(train_valid_test_dataset_dict["train"][:7]["labels"])
print(type(train_valid_test_dataset_dict["train"][:7]["labels"][0]))
print(train_valid_test_dataset_dict["train"][:7]["labels"][0])

<class 'list'>
[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x348 at 0x7F8AE220FD90>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x321 at 0x7F8AE2EEF3D0>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=443x428 at 0x7F8AE2EEF580>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=400x300 at 0x7F8AE2EEF640>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x375 at 0x7F8AE2EEFB20>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x352 at 0x7F8AE2EEFBB0>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=329x208 at 0x7F8AE2EEFCA0>]
<class 'list'>
[1, 0, 1, 1, 1, 0, 0]
<class 'int'>
1


In [ ]:
# train、validation、testデータ毎に写真と正解ラベルをそれぞれリストの形で取得
train_img_data_list = train_valid_test_dataset_dict["train"][:30]["image"]
train_label_value_list = train_valid_test_dataset_dict["train"][:30]["labels"]
valid_img_data_list = train_valid_test_dataset_dict["validation"][:6]["image"]
valid_label_value_list = train_valid_test_dataset_dict["validation"][:6]["labels"]
test_img_data_list = train_valid_test_dataset_dict["test"][:3]["image"]
test_label_value_list = train_valid_test_dataset_dict["test"][:3]["labels"]

In [ ]:
# train、validation、testデータをそれぞれdataset化
train_datasets = MyDataset(img_data_list=train_img_data_list,
                           label_value_list=train_label_value_list,
                           argumentation_method=train_argumentation)
valid_datasets = MyDataset(img_data_list=valid_img_data_list,
                           label_value_list=valid_label_value_list,
                           argumentation_method=valid_argumentation)
test_datasets = MyDataset(img_data_list=test_img_data_list,
                          label_value_list=test_label_value_list,
                          argumentation_method=test_argumentation)

In [ ]:
# train、validation、testデータのdatasetをそれぞれdataloader化
train_datasets_dataloader = torch.utils.data.DataLoader(dataset=train_datasets,
                                                        batch_size=8,
                                                        shuffle=True)
valid_datasets_dataloader = torch.utils.data.DataLoader(dataset=valid_datasets,
                                                        batch_size=8,
                                                        shuffle=True)
test_datasets_dataloader = torch.utils.data.DataLoader(dataset=test_datasets,
                                                       batch_size=8,
                                                       shuffle=True)

In [ ]:
print(len(train_datasets))
print(len(train_datasets_dataloader))
print(len(valid_datasets))
print(len(valid_datasets_dataloader))
print(len(test_datasets))
print(len(test_datasets_dataloader))

30
4
6
1
3
1


**VisionTransformerモデル**

In [ ]:
# VisionTransformerモデル(参考URL：https://qiita.com/zisui-sukitarou/items/d990a9630ff2c7f4abf2)
nn_model = ViT(dim=128,  # Embedding層での次元数
               image_size=224,  # 画像のサイズ(正方形の想定)
               patch_size=7,  # 画像を切り出すパッチのサイズ(パッチ数は、image_size/patch_sizeの2乗になる)
               num_classes=2,  # 出力層のパーセプトロン数(画像を分類する数)
               channels=3,  # チャンネル数でRGBの場合は3 # ここまではEmbedding層の設定のイメージ
               transformer=x_transformers.Encoder(dim=128,  # Transformer層での次元数 # ここからはTransformer層の設定のイメージ
                                                  depth=12,  # Transformer層の深さ
                                                  heads=8,  # Transformer層のMultiHeadAttentionの数
                                                  ff_glu=True,  # FeedForwardingNetworkでの活性化関数でGLUを使うかどうか
                                                  residual_attn=True)).to(device)  # ResidualAttentionBlockを使うかどうか # GPUへ
# 損失関数
loss_f = torch.nn.CrossEntropyLoss()
# 最適化アルゴリズム
optimizer = torch.optim.Adam(nn_model.parameters(),
                             lr=3e-5)

**学習**

In [ ]:
for epoch in range(5):
    print("*****{a}エポック目*****".format(a=epoch+1))
    # 学習
    epoch_loss_sum = 0
    train_epoch_loss_sum_list = []
    epoch_acc_sum = 0
    epoch_accuracy = 0
    train_epoch_accuracy_list = []
    nn_model.train()
    if os.path.isfile("./VisionTransformer_Classification_01.model"):
        nn_model.load_state_dict(torch.load("./VisionTransformer_Classification_01.model"))
    for img_data, label_value in tqdm.notebook.tqdm(train_datasets_dataloader):
        print(img_data.shape)
        img_data = img_data.to(device)  # GPUへ
        label_value = label_value.to(device)  # GPUへ
        output = nn_model(img_data)
        loss = loss_f(output, label_value)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss_sum = epoch_loss_sum + loss.item()
        output_predict = torch.argmax(output, dim=1).to("cpu")
        print("予測：", output_predict)
        answer = label_value.to("cpu")
        print("正解：", answer)
        acc = torch.sum(output_predict == answer).item()  # スカラー値のtensorをitem()でintやfloatとして取得
        print("正解数：", acc, " / ", len(output_predict))
        epoch_acc_sum = epoch_acc_sum + acc
    epoch_accuracy = epoch_acc_sum / len(train_datasets)
    print("{a}エポック目 学習loss：".format(a=epoch+1), epoch_loss_sum)
    train_epoch_loss_sum_list.append(epoch_loss_sum)
    print("{a}エポック目 学習acc：".format(a=epoch+1), epoch_accuracy*100 , "%")
    train_epoch_accuracy_list.append(epoch_accuracy)
    # 検証
    epoch_loss_sum = 0
    valid_epoch_loss_sum_list = []
    epoch_acc_sum = 0
    epoch_accuracy = 0
    valid_epoch_accuracy_list = []
    nn_model.eval()
    with torch.no_grad():
        for img_data, label_value in tqdm.notebook.tqdm(valid_datasets_dataloader):
            img_data = img_data.to(device)  # GPUへ
            label_value = label_value.to(device)  # GPUへ
            output = nn_model(img_data)
            loss = loss_f(output, label_value)
            epoch_loss_sum = epoch_loss_sum + loss.item()
            output_predict = torch.argmax(output, dim=1).to("cpu")
            answer = label_value.to("cpu")
            acc = torch.sum(output_predict == label_value).item()  # スカラー値のtensorをitem()でintやfloatとして取得
            epoch_acc_sum = epoch_acc_sum + acc
        epoch_accuracy = epoch_acc_sum / len(valid_datasets)
        print("{a}エポック目 検証loss：".format(a=epoch+1), epoch_loss_sum)
        valid_epoch_loss_sum_list.append(epoch_loss_sum)
        print("{a}エポック目 検証acc：".format(a=epoch+1), epoch_accuracy*100, "%")
        valid_epoch_accuracy_list.append(epoch_accuracy)
    # 学習で更新したパラメータを保存
    torch.save(nn_model.state_dict(), "./VisionTransformer_Classification_01.model")

*****1エポック目*****


  0%|          | 0/4 [00:00<?, ?it/s]

torch.Size([8, 3, 224, 224])
予測： tensor([1, 1, 1, 1, 1, 1, 1, 1])
正解： tensor([0, 1, 1, 1, 0, 0, 1, 0])
正解数： 4  /  8
torch.Size([8, 3, 224, 224])
予測： tensor([1, 1, 1, 1, 1, 1, 1, 1])
正解： tensor([0, 1, 1, 1, 1, 0, 1, 0])
正解数： 5  /  8
torch.Size([8, 3, 224, 224])
予測： tensor([1, 1, 1, 1, 1, 1, 1, 1])
正解： tensor([0, 0, 0, 1, 1, 0, 1, 0])
正解数： 3  /  8
torch.Size([6, 3, 224, 224])
予測： tensor([1, 1, 1, 1, 1, 1])
正解： tensor([1, 1, 0, 1, 0, 1])
正解数： 4  /  6
1エポック目 学習loss： 3.0810824036598206
1エポック目 学習acc： 53.333333333333336 %


  0%|          | 0/1 [00:00<?, ?it/s]

1エポック目 検証loss： 0.7227637767791748
1エポック目 検証acc： 33.33333333333333 %
*****2エポック目*****


...

4エポック目 検証loss： 0.7474479675292969
4エポック目 検証acc： 33.33333333333333 %
*****5エポック目*****


  0%|          | 0/4 [00:00<?, ?it/s]

torch.Size([8, 3, 224, 224])
予測： tensor([1, 1, 1, 1, 0, 1, 1, 1])
正解： tensor([0, 1, 1, 0, 1, 1, 1, 0])
正解数： 4  /  8
torch.Size([8, 3, 224, 224])
予測： tensor([1, 1, 1, 1, 1, 1, 1, 1])
正解： tensor([1, 0, 1, 1, 0, 0, 0, 0])
正解数： 3  /  8
torch.Size([8, 3, 224, 224])
予測： tensor([0, 1, 1, 0, 1, 1, 1, 1])
正解： tensor([0, 1, 1, 0, 0, 1, 0, 1])
正解数： 6  /  8
torch.Size([6, 3, 224, 224])
予測： tensor([1, 1, 1, 1, 1, 1])
正解： tensor([1, 1, 0, 1, 0, 1])
正解数： 4  /  6
5エポック目 学習loss： 2.6493571996688843
5エポック目 学習acc： 56.666666666666664 %


  0%|          | 0/1 [00:00<?, ?it/s]

5エポック目 検証loss： 0.7656672596931458
5エポック目 検証acc： 33.33333333333333 %


In [ ]:
# モデルの中身を表示
torchinfo.summary(nn_model,
                  input_size=(8, 3, 224, 224))  # モデルへ入力するデータの形状

# 1024は、画像サイズが224×224の1枚の画像を、7×7のパッチに分割するので、224÷7=32から、32×32=1024個のパッチになり、それが時系列になるため
# 128は、Embedding層での次元数を128で指定しているため

Layer (type:depth-idx)                             Output Shape              Param #
ViT                                                [8, 2]                    131,328
├─Sequential: 1-1                                  [8, 1024, 128]            --
│    └─Rearrange: 2-1                              [8, 1024, 147]            --
│    └─LayerNorm: 2-2                              [8, 1024, 147]            294
│    └─Linear: 2-3                                 [8, 1024, 128]            18,944
│    └─LayerNorm: 2-4                              [8, 1024, 128]            256
├─Encoder: 1-2                                     [8, 1025, 128]            --
│    └─ModuleList: 2-5                             --                        --
│    │    └─ModuleList: 3-1                        --                        262,400
│    │    └─ModuleList: 3-2                        --                        198,016
│    │    └─ModuleList: 3-3                        --                        262,400
│    │   

**事前学習済みモデルを使ってみる**

In [ ]:
# timmで事前学習済みのモデル名を取得して、その一部のモデル名を表示
timm_pretrained_model_list = timm.list_models(pretrained=True)
pprint.pprint(timm_pretrained_model_list[:15])

['bat_resnext26ts.ch_in1k',
 'beit_base_patch16_224.in22k_ft_in22k',
 'beit_base_patch16_224.in22k_ft_in22k_in1k',
 'beit_base_patch16_384.in22k_ft_in22k_in1k',
 'beit_large_patch16_224.in22k_ft_in22k',
 'beit_large_patch16_224.in22k_ft_in22k_in1k',
 'beit_large_patch16_384.in22k_ft_in22k_in1k',
 'beit_large_patch16_512.in22k_ft_in22k_in1k',
 'beitv2_base_patch16_224.in1k_ft_in1k',
 'beitv2_base_patch16_224.in1k_ft_in22k',
 'beitv2_base_patch16_224.in1k_ft_in22k_in1k',
 'beitv2_large_patch16_224.in1k_ft_in1k',
 'beitv2_large_patch16_224.in1k_ft_in22k',
 'beitv2_large_patch16_224.in1k_ft_in22k_in1k',
 'botnet26t_256.c1_in1k']


In [ ]:
# モデルは、事前学習済みモデル「vit_small_patch16_224」を呼び出す
nn_model = timm.create_model(model_name="vit_small_patch16_224",
                             pretrained=True,
                             num_classes=2).to(device)
loss_f = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(nn_model.parameters(),
                             lr=3e-5)

model.safetensors:   0%|          | 0.00/88.2M [00:00<?, ?B/s]

In [ ]:
for epoch in range(5):
    print("*****{a}エポック目*****".format(a=epoch+1))
    # 学習(Fine Tuning)
    epoch_loss_sum = 0
    train_epoch_loss_sum_list = []
    epoch_acc_sum = 0
    epoch_accuracy = 0
    train_epoch_accuracy_list = []
    nn_model.train()
    if os.path.isfile("./VisionTransformer_PreTrained_Classification_01.model"):
        nn_model.load_state_dict(torch.load("./VisionTransformer_PreTrained_Classification_01.model"))
    for img_data, label_value in tqdm.notebook.tqdm(train_datasets_dataloader):
        print(img_data.shape)
        img_data = img_data.to(device)  # GPUへ
        label_value = label_value.to(device)  # GPUへ
        output = nn_model(img_data)
        loss = loss_f(output, label_value)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss_sum = epoch_loss_sum + loss.item()
        output_predict = torch.argmax(output, dim=1).to("cpu")
        print("予測：", output_predict)
        answer = label_value.to("cpu")
        print("正解：", answer)
        acc = torch.sum(output_predict == answer).item()  # スカラー値のtensorをitem()でintやfloatとして取得
        print("正解数：", acc, " / ", len(output_predict))
        epoch_acc_sum = epoch_acc_sum + acc
    epoch_accuracy = epoch_acc_sum / len(train_datasets)
    print("{a}エポック目 学習loss：".format(a=epoch+1), epoch_loss_sum)
    train_epoch_loss_sum_list.append(epoch_loss_sum)
    print("{a}エポック目 学習acc：".format(a=epoch+1), epoch_accuracy*100 , "%")
    train_epoch_accuracy_list.append(epoch_accuracy)
    # 検証
    epoch_loss_sum = 0
    valid_epoch_loss_sum_list = []
    epoch_acc_sum = 0
    epoch_accuracy = 0
    valid_epoch_accuracy_list = []
    nn_model.eval()
    with torch.no_grad():
        for img_data, label_value in tqdm.notebook.tqdm(valid_datasets_dataloader):
            img_data = img_data.to(device)  # GPUへ
            label_value = label_value.to(device)  # GPUへ
            output = nn_model(img_data)
            loss = loss_f(output, label_value)
            epoch_loss_sum = epoch_loss_sum + loss.item()
            output_predict = torch.argmax(output, dim=1).to("cpu")
            answer = label_value.to("cpu")
            acc = torch.sum(output_predict == label_value).item()  # スカラー値のtensorをitem()でintやfloatとして取得
            epoch_acc_sum = epoch_acc_sum + acc
        epoch_accuracy = epoch_acc_sum / len(valid_datasets)
        print("{a}エポック目 検証loss：".format(a=epoch+1), epoch_loss_sum)
        valid_epoch_loss_sum_list.append(epoch_loss_sum)
        print("{a}エポック目 検証acc：".format(a=epoch+1), epoch_accuracy*100, "%")
        valid_epoch_accuracy_list.append(epoch_accuracy)
    # 学習(Fine Tuning)で更新したパラメータを保存
    torch.save(nn_model.state_dict(), "./VisionTransformer_PreTrained_Classification_01.model")

*****1エポック目*****


  0%|          | 0/4 [00:00<?, ?it/s]

torch.Size([8, 3, 224, 224])
予測： tensor([0, 1, 0, 0, 1, 0, 1, 1])
正解： tensor([1, 1, 1, 0, 1, 1, 1, 1])
正解数： 5  /  8
torch.Size([8, 3, 224, 224])
予測： tensor([1, 1, 1, 1, 0, 0, 0, 0])
正解： tensor([1, 1, 1, 1, 0, 0, 0, 0])
正解数： 8  /  8
torch.Size([8, 3, 224, 224])
予測： tensor([0, 1, 0, 0, 0, 0, 1, 0])
正解： tensor([0, 1, 0, 0, 1, 0, 1, 0])
正解数： 7  /  8
torch.Size([6, 3, 224, 224])
予測： tensor([0, 0, 0, 0, 0, 1])
正解： tensor([0, 1, 0, 0, 0, 1])
正解数： 5  /  6
1エポック目 学習loss： 0.9421654492616653
1エポック目 学習acc： 83.33333333333334 %


  0%|          | 0/1 [00:00<?, ?it/s]

1エポック目 検証loss： 0.21278314292430878
1エポック目 検証acc： 100.0 %
*****2エポック目*****


...

4エポック目 検証loss： 0.20055566728115082
4エポック目 検証acc： 83.33333333333334 %
*****5エポック目*****


  0%|          | 0/4 [00:00<?, ?it/s]

torch.Size([8, 3, 224, 224])
予測： tensor([0, 0, 0, 1, 1, 0, 1, 1])
正解： tensor([0, 0, 0, 1, 1, 0, 1, 1])
正解数： 8  /  8
torch.Size([8, 3, 224, 224])
予測： tensor([1, 0, 1, 1, 0, 1, 1, 0])
正解： tensor([1, 0, 1, 1, 0, 1, 1, 0])
正解数： 8  /  8
torch.Size([8, 3, 224, 224])
予測： tensor([0, 0, 1, 0, 1, 1, 1, 1])
正解： tensor([0, 0, 1, 0, 1, 1, 1, 1])
正解数： 8  /  8
torch.Size([6, 3, 224, 224])
予測： tensor([0, 0, 1, 0, 0, 1])
正解： tensor([0, 0, 1, 0, 0, 1])
正解数： 6  /  6
5エポック目 学習loss： 0.0702239042147994
5エポック目 学習acc： 100.0 %


  0%|          | 0/1 [00:00<?, ?it/s]

5エポック目 検証loss： 0.1540786772966385
5エポック目 検証acc： 83.33333333333334 %


In [ ]:
# モデルの中身を表示
torchinfo.summary(nn_model,
                  input_size=(8, 3, 224, 224))  # モデルへ入力するデータの形状

Layer (type:depth-idx)                   Output Shape              Param #
VisionTransformer                        [8, 2]                    76,032
├─PatchEmbed: 1-1                        [8, 196, 384]             --
│    └─Conv2d: 2-1                       [8, 384, 14, 14]          295,296
│    └─Identity: 2-2                     [8, 196, 384]             --
├─Dropout: 1-2                           [8, 197, 384]             --
├─Identity: 1-3                          [8, 197, 384]             --
├─Identity: 1-4                          [8, 197, 384]             --
├─Sequential: 1-5                        [8, 197, 384]             --
│    └─Block: 2-3                        [8, 197, 384]             --
│    │    └─LayerNorm: 3-1               [8, 197, 384]             768
│    │    └─Attention: 3-2               [8, 197, 384]             591,360
│    │    └─Identity: 3-3                [8, 197, 384]             --
│    │    └─Identity: 3-4                [8, 197, 384]             --
